In [1]:
import numpy as np 
import pandas as pd 
from recipes import Recipes

In [2]:
cuukin = Recipes()

In [3]:
cuukin.import_data(listings_folder='listings', training_folder='training')

In [4]:
def find_technique_name(id_list):
    result = []
    for id in id_list:
        result.append(cuukin.listings['techniques']['name'][id])
    return result

cuukin.training_data['recipe_techniques'] = cuukin.training_data['recipe_techniques'].assign(technique_name = find_technique_name(cuukin.training_data['recipe_techniques']['technique_id']))

In [5]:
cuukin.training_data['recipe_techniques'].head(3)

,recipe_id,technique_id,created_at,updated_at,technique_name
id,,,,,
34,15,2,2021-04-27 21:03:26.635000+00:00,2021-04-27 21:03:26.635000+00:00,Dicing 1
33,15,51,2021-04-27 21:02:56.154000+00:00,2021-04-27 21:02:56.154000+00:00,Boiling
32,15,52,2021-04-27 21:02:21.199000+00:00,2021-04-27 21:02:21.199000+00:00,Simmering


In [6]:
cuukin.listings['techniques'].head(3)

,name,score_1,score_2,score_3,badge_id,created_at,updated_at
id,,,,,,,
67,Measuring,15,2,0,36,2021-03-13 17:53:26.128000+00:00,2021-03-13 17:53:26.128000+00:00
66,Plating,15,2,0,40,2021-03-13 17:53:26.116000+00:00,2021-03-13 17:53:26.116000+00:00
65,Smoking,15,2,0,39,2021-03-13 17:53:26.107000+00:00,2021-03-13 17:53:26.107000+00:00


In [7]:
cathegories = cuukin.listings['techniques']['name']
dummies = pd.get_dummies(cuukin.training_data['recipe_techniques']['technique_name'])
missing_columns = list(set(cathegories)- set(dummies))

In [8]:
number_rows = len(cuukin.training_data['recipe_techniques'].index)
missing_data_frame = pd.DataFrame(0, index=np.arange(1,number_rows+1), columns=missing_columns)
missing_data_frame.head(3)

,Deglazing 1,Steaming,Deep Frying,Stir Frying 1,Mincing 2,Freezing,Whisking 1,Water Bathing,Flambing,Sauteeing 2,...,Seasoning 2,Stir Frying 2,Roasting,Pickling,Kneading,Blending,Air Frying,Double Boiler,Squeezing,Tenderizing
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
merged = pd.concat([cuukin.training_data['recipe_techniques'], dummies, missing_data_frame], axis='columns')
merged.drop(columns=['created_at', 'updated_at', 'technique_name', 'technique_id'], inplace=True)
merged.head(3)

,recipe_id,Baking 1,Boiling,Dicing 1,Microwaving,Mixing,Pan Frying 1,Plating,Roughly Chopping,Salting 1,...,Seasoning 2,Stir Frying 2,Roasting,Pickling,Kneading,Blending,Air Frying,Double Boiler,Squeezing,Tenderizing
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
grouped = merged.groupby(['recipe_id'])
grouped = grouped.sum()
cuukin.training_data["recipe_techniques_vector"] = grouped

cuukin.training_data["recipe_techniques_vector"].head()

,Baking 1,Boiling,Dicing 1,Microwaving,Mixing,Pan Frying 1,Plating,Roughly Chopping,Salting 1,Sauteeing 1,...,Seasoning 2,Stir Frying 2,Roasting,Pickling,Kneading,Blending,Air Frying,Double Boiler,Squeezing,Tenderizing
recipe_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
